# Crashes to JSON

Use `Data Query Tool` to create a new export from UCONN Crashes (Town=Hartford, last five years): https://www.ctcrash.uconn.edu/QueryTool2.action

* Since the tool does not let you export large datasets (over 90,000 persons), export multiple times for different time periods and specify all export IDs in the `exports` list below.

This notebook combines data from two tables (0 and 2) into a small JSON file, which is an array of objects of thie following kind:
```js
{
    "id": 770826,    // crash id (used to pull the diagram)
    "x": 41.73936,   // latitude
    "y": -72.66404,  // longitude
    "d": 1593820800, // timestamp
    "p": 0,          // involves a pedestrian? (1=true, 0=false)
    "c": 0,          // involves a cyclist? (1=true, 0=false)
    "r": 1,          // route class (1=Interstate, 2=USRoute, 3=State, 4=Local)
    "s": 0,          // crash severity (O=no injuries, A=any injury, K=fatal)
}
```

Note: the timestamp `d` needs to be multiplied by `100,000` for a proper JS timestamp.

In [15]:
import pandas as pd

In [16]:
exports = ['110308', '81849']

### Import roles from table 2

In [17]:
roles_dfs = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_2.csv',
        skiprows=1,
        usecols=['CrashId', 'Person Type Text Format']
    )
    roles_dfs.append( df )

roles = pd.concat(roles_dfs).groupby(['CrashId'])['Person Type Text Format'].unique()

### Import crashes from table 0

In [28]:
crashes_dfs = []

for export in exports:
    df = pd.read_csv(
        f'export_{export}/export_{export}_0.csv',
        skiprows=1,
        usecols=['CrashId', 'Latitude', 'Longitude', 'Date Of Crash',
                 'Time of Crash', 'Crash Severity', 'Route Class']
    )
    crashes_dfs.append( df )
    
crashes = pd.concat(crashes_dfs)

In [30]:
test = crashes.copy()

In [37]:
test['Mo'] = test['Date Of Crash'].apply(lambda x: x[:7])
test.Mo.value_counts().sort_index().tail(10)

2020-01    595
2020-02    589
2020-03    466
2020-04    240
2020-05    326
2020-06    438
2020-07    452
2020-08    208
2020-09     73
2020-10     18
Name: Mo, dtype: int64

### Merge data and save to .json and .csv

In [19]:
crashes.head()

,CrashId,Latitude,Longitude,Date Of Crash,Time of Crash,Crash Severity,Route Class
0,580921,41.746565,-72.659791,2019-01-09,15:29:00,O,1
1,585498,41.764155,-72.693364,2019-01-22,09:31:00,A,1
2,586562,41.730579,-72.660844,2019-01-14,16:30:00,O,1
3,588118,41.746148,-72.660153,2019-01-25,15:48:00,A,1
4,595170,41.769291,-72.668473,2019-01-01,03:49:00,A,1


In [20]:
# Rename columns to make JSON output smaller
crashes.columns = ['id', 'x', 'y', 'd', 't', 's', 'r']

# Transform timestamps
crashes.d = crashes.d.apply(lambda x: int(pd.to_datetime(x).timestamp() / 100))
crashes.t = crashes.t.apply(lambda x: x[:-3])

# Set up pedestrian and cyclist flags
crashes['p'] = crashes.id.apply(lambda x: 1 if x in roles and 'Pedestrian' in roles[x] else 0)
crashes['c'] = crashes.id.apply(lambda x: 1 if x in roles and 'Bicyclist' in roles[x] else 0)

In [22]:
# Save as JSON
crashes.to_json('./crashes.json', orient='records', double_precision=5)

# Save as CSV
crashes.to_csv('crashes.csv', index=False)